In [1]:
import pandas as pd
from os import rename
from datetime import datetime
from all_p import ClusterFile
from os.path import join
from os import walk

In [15]:
class Watch:
    table = None
    def_name = '/home/rt2122/watch/last.csv'
    def_dir = '/home/rt2122/watch/'
    states = ('no', 'ne', 'cl', 'grizym')
    changed = False
    vrb = False
    def __new__(obj):
        if not hasattr(obj, 'instance'):
            obj.instance=super(Watch, obj).__new__(obj);
        return obj.instance;
    
    def __init__(self):
        self.table = pd.read_csv(self.def_name, index_col='index')


    def drop(self):
        #~~~~~~~~~~~~~~~~~#
        def name_backup():
            now = datetime.now()
            return 'back%d.%d_%d:%d:%d.csv' % (now.month, now.day, now.hour, now.minute, now.second)
        #~~~~~~~~~~~~~~~~~#
        print('Drop')
        if self.changed:
            rename(self.def_name, join(self.def_dir, name_backup()))
            self.table.to_csv(self.def_name)
        
    def get_patch_idx(self, id_list, id_patch):
        idx = self.table[self.table['id_list'] == id_list][self.table['id_patch'] == id_patch].index
        if len(idx) > 0:
            return idx[0]
        return None
    
    def add_patch(self, cf, idx, i=None, id_list=None, from_list=True):
        self.changed = True
        if from_list:
            self.table['id_list'].loc[idx] = id_list
            self.table['id_patch'].loc[idx] = i
        else:
            self.table['state'].loc[idx] = cf.params['state']
            self.table['id_list'].loc[idx] = cf.params['id_list']
            self.table['id_patch'].loc[idx] = cf.params['id_patch']
            
        self.table['ra'].loc[idx] = cf.params['ra']
        self.table['dec'].loc[idx] = cf.params['dec']
        self.table['inpix'].loc[idx] = cf.params['inpix']
        
        loc = 'from list' if from_list else 'from name'
        
        if self.vrb:
            print('Added patch %s list #%d patch #%d' % (loc, self.table['id_list'].iloc[idx],
                                                        self.table['id_patch'].iloc[idx]))
    
    def update_state(self, cf, idx):
        new_state = cf.params['state']
        if new_state in self.states:
            new_state_i = self.states.index(new_state)
            old_state = str(self.table['state'].iloc[idx])
            old_state_i = self.states.index(self.table['state'].iloc[idx])
            if new_state_i > old_state_i:
                self.table['state'].loc[idx] = new_state
                if self.vrb:
                    print('From %d list patch #%d changed from %s to %s' % (cf.params['id_list'], 
                                                                           cf.params['id_patch'], 
                                                                           old_state, new_state))
                
    def update(self, fname, data=None):
        cf = ClusterFile(fname)
        if cf.params['typ'] == 'cen':
            if data is None:
                return
            id_list = cf.params['id_list']
            for i in range(data.shape[0]):
                idx = self.get_patch(id_list, idx, i, id_list)
                if idx is None:
                    idx = self.table.shape[0]
                    self.add_patch(cf, idx, i, id_list)
        elif cf.params['typ'] == 'dat' or cf.params['typ'] == 'm' or cf.params['typ'] == 'f':
            
            if not (data is None) and self.vrb:
                print('I dont need data for patch!')
            idx = self.get_patch_idx(cf.params['id_list'], cf.params['id_patch'])
            if idx is None:
                idx = self.table.shape[0]
                self.add_patch(cf, idx, from_list=False)
            else:
                self.update_state(cf, idx)
            
            

In [31]:
watch = Watch()

In [32]:
files = next(walk('/home/rt2122/data/train/filters/'))[-1]

In [33]:
watch.update(files[0])

Added patch from name list #1 patch #2


/home/rt2122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [34]:
for f in files:
    watch.update(f)

Added patch from name list #1 patch #2
From 0 list patch #17 changed from cl to grizym
From 0 list patch #18 changed from cl to grizym
Added patch from name list #1 patch #19
From 0 list patch #10 changed from cl to grizym
Added patch from name list #1 patch #9
Added patch from name list #1 patch #19
Added patch from name list #1 patch #16
Added patch from name list #1 patch #2
Added patch from name list #1 patch #18
Added patch from name list #1 patch #14
From 0 list patch #3 changed from cl to grizym
Added patch from name list #1 patch #16
Added patch from name list #1 patch #4
Added patch from name list #1 patch #17
From 0 list patch #8 changed from cl to grizym
Added patch from name list #1 patch #10
From 0 list patch #16 changed from cl to grizym
Added patch from name list #1 patch #3
Added patch from name list #1 patch #9
Added patch from name list #1 patch #4
From 0 list patch #12 changed from cl to grizym
From 0 list patch #9 changed from cl to grizym
Added patch from name list

/home/rt2122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [27]:
watch.table.head()

,id_patch,id_list,ra,dec,inpix,state
index,,,,,,
0,0,0,68.149720,67.343782,4,cl
1,1,0,95.659138,66.964809,4,cl
2,2,0,91.627482,67.480008,4,cl
3,3,0,68.574847,86.600743,4,cl
4,4,0,93.645251,74.511402,4,cl


In [17]:
watch.vrb = True

In [7]:
for f in files:
    watch.update(f)

In [140]:
watch.changed=True
watch.drop()

Drop


In [116]:
watch.states

('no', 'ne', 'cl', 'grizym')

In [24]:
watch.table[watch.table['id_list'] == '1'].shape

/home/rt2122/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/home/rt2122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(0, 6)

In [21]:
set(watch.table['id_list'])

{0, 1}

In [30]:
watch.table.iloc[47:]

,id_patch,id_list,ra,dec,inpix,state
index,,,,,,
47,47,0,120.657084,74.117886,4,no
48,48,0,104.212103,86.241356,4,no
49,49,0,104.135893,70.532809,4,no


In [35]:
watch.table.shape

(50, 6)